In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas
od.download(
    "https://www.kaggle.com/datasets/niraliivaghani/chatbot-dataset")

Skipping, found downloaded files in "./chatbot-dataset" (use force=True to force download)


In [ ]:
import tensorflow as tf
import numpy as np


In [ ]:
import json

# Opening JSON file
f = open('/content/chatbot-dataset/intents.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Iterating through the json
# list
for i in data['intents']:
    print(i)

{'tag': 'greeting', 'patterns': ['Hi', 'How are you?', 'Is anyone there?', 'Hello', 'Good day', "What's up", 'how are ya', 'heyy', 'whatsup', '??? ??? ??'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}
{'tag': 'goodbye', 'patterns': ['cya', 'see you', 'bye bye', 'See you later', 'Goodbye', 'I am Leaving', 'Bye', 'Have a Good day', 'talk to you later', 'ttyl', 'i got to go', 'gtg'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!', 'Come back soon'], 'context_set': ''}
{'tag': 'creator', 'patterns': ['what is the name of your developers', 'what is the name of your creators', 'what is the name of the developers', 'what is the name of the creators', 'who created you', 'your developers', 'your creators', 'who are your developers', 'developers', 'you are made by', 'you are made by whom', 'who created you', 'who create you', 'creators', 'who made you', 'who designed you'], 'responses': ['College students'], 'context_s

In [ ]:
# tf.keras.preprocessing.text.tokenizer_from_json(
#     json_string
# )
patterns = []
tags = []
labels=[]
train_data = []
train_labels =[]
for intent in data["intents"]:
  tags.append(intent["tag"])
  for pattern in intent["patterns"]:
    train_data.append(pattern)
    labels.append(intent["tag"])
      # token = tf.strings.split(pattern)
      # vocab.extend(token)
for tag in tags:
  label = tags.index(tag)
  for x in labels:
    if x==tag:
      train_labels.append(label)
#   # train_labels.append(labels)
train_data[:10],train_labels[-10:]


(['Hi',
  'How are you?',
  'Is anyone there?',
  'Hello',
  'Good day',
  "What's up",
  'how are ya',
  'heyy',
  'whatsup',
  '??? ??? ??'],
 [36, 36, 36, 36, 36, 36, 36, 37, 37, 37])

In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    idf_weights=None,
    sparse=False,
    ragged=False,
    encoding='utf-8',
)

In [ ]:
# train_data = np.array(train_data)
# train_data[:10]

# train_data = train_data.reshape(-1, 1)

In [ ]:
text_vectorizer.adapt(train_data)

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:10]


['', '[UNK]', 'is', 'college', 'what', 'the', 'you', 'of', 'in', 'for']

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim = len(words_in_vocab),
                             output_dim = 128,
                             input_length = len(words_in_vocab))
embedding


In [ ]:
import random
random_sentence = random.choice(train_data)
embedded_sentence = embedding(text_vectorizer([random_sentence]))
embedded_sentence
# random_sentence

<tf.Tensor: shape=(1, 1, 128), dtype=float32, numpy=
array([[[-0.01764596,  0.01279861, -0.04840164, -0.014346  ,
          0.0459368 ,  0.01316785,  0.02003806,  0.04623621,
         -0.00230287, -0.00612942, -0.0152493 ,  0.03035747,
          0.00965834,  0.00318814, -0.04841471, -0.03983925,
         -0.03442036,  0.03335447,  0.03298248,  0.030301  ,
          0.0175735 , -0.04306504,  0.02825497,  0.00847657,
         -0.0223417 ,  0.03150903, -0.04611404, -0.02032905,
          0.01387571, -0.04206102,  0.04875419, -0.04558026,
         -0.02930919,  0.01440806, -0.01354546,  0.04076506,
         -0.01207333, -0.02365447, -0.00646137,  0.0159001 ,
         -0.02081871,  0.04511055, -0.00182177, -0.01468064,
          0.01190764, -0.01090554,  0.02671457,  0.0148907 ,
         -0.00634208,  0.02691196,  0.01245791,  0.01774789,
         -0.00672439,  0.02795471,  0.04115841, -0.03089377,
          0.02668503,  0.02682826, -0.04098777,  0.03420032,
          0.00917574,  0.0441090

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(256,return_sequences = True)(x)
x = layers.LSTM(256)(x)
x = layers.Dense(64)(x)
outputs = layers.Dense(38,activation="softmax")(x)
model = tf.keras.Model(inputs,outputs)

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model.fit(train_data,train_labels,epochs = 10)

Epoch 1/10
13/13 [==============================] - 7s 88ms/step - loss: 3.3779 - accuracy: 0.1259
Epoch 2/10
13/13 [==============================] - 1s 89ms/step - loss: 2.6515 - accuracy: 0.2222
Epoch 3/10
13/13 [==============================] - 1s 88ms/step - loss: 1.9436 - accuracy: 0.4469
Epoch 4/10
13/13 [==============================] - 1s 86ms/step - loss: 1.2676 - accuracy: 0.6642
Epoch 5/10
13/13 [==============================] - 1s 86ms/step - loss: 0.8250 - accuracy: 0.7654
Epoch 6/10
13/13 [==============================] - 1s 85ms/step - loss: 0.5303 - accuracy: 0.8469
Epoch 7/10
13/13 [==============================] - 1s 109ms/step - loss: 0.4293 - accuracy: 0.8889
Epoch 8/10
13/13 [==============================] - 2s 136ms/step - loss: 0.2862 - accuracy: 0.9136
Epoch 9/10
13/13 [==============================] - 1s 88ms/step - loss: 0.2539 - accuracy: 0.9284
Epoch 10/10
13/13 [==============================] - 1s 84ms/step - loss: 0.2232 - accuracy: 0.9407


In [ ]:
labels[34]

'creator'

In [ ]:
tf.squeeze(tf.round(model.predict(["How are you?"])))

1/1 [==============================] - 1s 951ms/step


<tf.Tensor: shape=(38,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)>

In [ ]:
x = model.predict(["May I know your name?"])

1/1 [==============================] - 1s 962ms/step


In [ ]:
x.argmax(1)

array([2])

In [ ]:
tags[2]

'creator'

In [ ]:
train_labels[4]

0